In [ ]:
import os
import numpy as np
import pandas as pd
import pandas_ta as ta
from pathlib import Path
from datetime import datetime


downloads_path = str(Path.home() / "Downloads")
tv_exp_fn = f'{downloads_path}\\BINANCE_BTCUSDT.P, 5.csv'
tst_fn = f'{downloads_path}\\tst.csv'

N = 4
per = 144
mult = 1.414
modeLag = False
modeFast = False
beta = 0.00503274062721631
alpha = 0.0954202818890498
lag = 17.875
m__x = 0.90457971811095
const_1 = 0.00008290157486616984


tv_df = pd.read_csv(tv_exp_fn)
os.remove(tv_exp_fn)
open_price = list(tv_df['open'])
high_price = list(tv_df['high'])
low_price = list(tv_df['low'])
close_price = list(tv_df['close'])

m_src = list(ta.hlc3(tv_df['high'], tv_df['low'], tv_df['close']))
tv__f = list(tv_df['tv__f'])
m_srcdata = m_src
m_trdata = [max([high_price[idx] - low_price[idx], abs(high_price[idx] - close_price[idx-1]), abs(low_price[idx] - close_price[idx-1])]) for idx in range(len(open_price))]
_i = N
_a = alpha
_s = m_srcdata
m__f = [0.0]*len(open_price)
m_m2 = (36 if _i == 9 else 28 if _i == 8 else 21 if _i == 7 else 15 if _i == 6 else 10 if _i == 5 else 6 if _i == 4 else 3 if _i == 3 else 1 if _i == 2 else 0)
m_m3 = (84 if _i == 9 else 56 if _i == 8 else 35 if _i == 7 else 20 if _i == 6 else 10 if _i == 5 else 4 if _i == 4 else 1 if _i == 3 else 0)
m__m4 = (126 if _i == 9 else 70 if _i == 8 else 35 if _i == 7 else 15 if _i == 6 else 5 if _i == 5 else 1 if _i == 4 else 0)
m__m5 = (126 if _i == 9 else 56 if _i == 8 else 21 if _i == 7 else 6 if _i == 6 else 1 if _i == 5 else 0)
m__m6 = (84 if _i == 9 else 28 if _i == 8 else 7 if _i == 7 else 1 if _i == 6 else 0)
m__m7 = (36 if _i == 9 else 8 if _i == 8 else 1 if _i == 7 else 0)
m__m8 = (9 if _i == 9 else 1 if _i == 8 else 0)
m__m9 = 1 if _i == 9 else 0

for idx in range(len(m_src)):
    m__f[idx] = (const_1 * m_src[idx]) + (_i * m__x ) * m__f[idx-1]







exp_df = pd.DataFrame()
exp_df['tv__f'] = tv__f
exp_df['m__f'] = m__f
exp_df['chk'] = [f"=ROUND(A{idx+2},1)=ROUND(B{idx+2},1)" for idx in range(len(open_price))]
exp_df['false_count'] = [f"=COUNTIF(C2:C{len(open_price)}, FALSE)"]+['']*(len(open_price)-1)
exp_df.to_csv(tst_fn, index=False)
os.system(f"start EXCEL.EXE {tst_fn}")


In [ ]:

import os
import numpy as np
import pandas as pd
import pandas_ta as ta
from pathlib import Path
from datetime import datetime


downloads_path = str(Path.home() / "Downloads")
tv_exp_fn = f'{downloads_path}\\BINANCE_BTCUSDT.P, 120.csv'
tst_fn = f'{downloads_path}\\tst.csv'

tv_df = pd.read_csv(tv_exp_fn)

tm = list(tv_df['time'])

open_price = list(tv_df['open'])
high_price = list(tv_df['high'])
low_price = list(tv_df['low'])
close_price = list(tv_df['close'])
hband = list(tv_df['hband'])
lband = list(tv_df['lband'])
sig = ['NS']*len(open_price)
f_sig = ['']*len(open_price)
last_sig = ''
for idx in range(len(open_price)):
    if low_price[idx] > hband[idx]:
        sig[idx] = 'L'
        if last_sig != 'L':
            f_sig[idx] = 'L'
            last_sig = 'L'
    if high_price[idx]  < lband[idx]:
        sig[idx] = 'S'
        if last_sig != 'S':
            f_sig[idx] = 'S'
            last_sig = 'S'

exp_df = pd.DataFrame()
exp_df['time'] = tm
exp_df['open'] = open_price
exp_df['high'] = high_price
exp_df['low'] = low_price
exp_df['close'] = close_price
exp_df['hband'] = hband
exp_df['lband'] = lband
exp_df['sig'] = sig
exp_df['f_sig'] = f_sig

exp_df.to_csv(tst_fn, index=False)
# os.system(f"start EXCEL.EXE {tst_fn}")

lng_idxs = []
srt_idxs = []

for idx, val in enumerate(f_sig):
    if val == 'L':
        lng_idxs.append(idx)
    if val == 'S':
        srt_idxs.append(idx)

lng_trds = []
srt_trds = []
target = 1
for idx, lng_idxs_val in enumerate(lng_idxs[:-1]):
    ep = open_price[lng_idxs_val]
    xp = open_price[srt_idxs[idx]]
    mx = max(high_price[lng_idxs_val:srt_idxs[idx]])
    mn = min(high_price[lng_idxs_val:srt_idxs[idx]])
    pnl = round((xp*100/ep)-100, 2)
    run_up = round((mx*100/ep)-100, 2)
    lng_trds.append(
        {
            'type': 'L',
            'entry_idx': lng_idxs_val,
            'entry_time': tm[lng_idxs_val],
            'entry_price': ep,
            'exit_idx': srt_idxs[idx],
            'exit_time': tm[srt_idxs[idx]],
            'exit_price': xp,
            'PnL': pnl,
            'run_up': run_up,
            # 'trgt': target if run_up >= target else pnl
        }
    )

for idx, srt_idxs_val in enumerate(srt_idxs[:-1]):
    xp = open_price[lng_idxs[idx + 1]]
    ep = open_price[srt_idxs_val]
    mn = min(low_price[srt_idxs_val:lng_idxs[idx + 1]])
    pnl = round((ep*100/xp)-100, 2)
    run_up = round((ep*100/mn)-100, 2)
    srt_trds.append(
        {
            'type': 'S',
            'entry_idx': srt_idxs_val,
            'entry_time': tm[srt_idxs_val],
            'entry_price': ep,
            'exit_idx': lng_idxs[idx + 1],
            'exit_time': tm[lng_idxs[idx + 1]],
            'exit_price': xp,
            'PnL': pnl,
            'run_up': run_up,
            # 'trgt': target if run_up >= target else pnl
        }
    )


all_trades_lst = []
for idx in range(len(srt_trds)):
    all_trades_lst.append(lng_trds[idx])
    all_trades_lst.append(srt_trds[idx])

all_trades_df = pd.DataFrame(all_trades_lst)
# =IF(I2>=$K$1,$K$1,H2)
all_trades_df['trgt'] = [f'=IF(I{i+2}>=$K$1,$K$1,H{i+2})' for i in range(len(all_trades_df))]
avg = round(all_trades_df["run_up"].mean(), 2)
print('avg: ', avg)
all_trades_df[f'{avg}'] = ['' for i in range(len(all_trades_df))]
dur = tv_exp_fn.split(', ')[1].split('.')[0]
all_trades_df.to_csv(f'{downloads_path}\\all_trades_df_{dur}.csv', index=False)


In [1]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG


class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()


bt = Backtest(GOOG, SmaCross,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
bt.plot()

c:\Users\Dev1\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Dev1\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\Dev1\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\Dev1\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using th

GridPlot(id='p1325', ...)